In [1]:
# Run the stored data from the Data_Preparation.ipynb
%run ..//Data_Preparation.ipynb

Data types in 'indu' dataframe:
Time       datetime64[ns]
indu_Q1           float64
indu_Q2           float64
indu_Q3           float64
indu_Q4           float64
indu_Q5           float64
indu_Q7           float64
dtype: object

Data types in 'cons' dataframe:
Time        datetime64[ns]
cons_Q1            float64
cons_Q2            float64
cons_Q3            float64
cons_Q4            float64
cons_Q5            float64
cons_Q6            float64
cons_Q7            float64
cons_Q8            float64
cons_Q9            float64
cons_Q10           float64
cons_Q11           float64
cons_Q12           float64
dtype: object

Data types in 'gdp' dataframe:
Time     datetime64[ns]
Value           float64
dtype: object
indu_Q1           float64
indu_Q2           float64
indu_Q3           float64
indu_Q4           float64
indu_Q5           float64
indu_Q7           float64
Time       datetime64[ns]
dtype: object
cons_Q1            float64
cons_Q2            float64
cons_Q3            float64
con

In [2]:
#Split the data into training and testing data
indu_train = indu[(indu['Time'] >= '1990-01-01') & (indu['Time'] < '2020-01-01')]
cons_train = cons[(cons['Time'] >= '1990-01-01') & (cons['Time'] < '2020-01-01')]
gdp_train = gdp[(gdp['Time'] >= '1990-01-01') & (gdp['Time'] < '2020-01-01')]

indu_test = indu[(indu['Time'] >= '2020-01-01') & (indu['Time'] < '2024-01-01')]
cons_test = cons[(cons['Time'] >= '2020-01-01') & (cons['Time'] < '2024-01-01')]
gdp_test = gdp[(gdp['Time'] >= '2020-01-01') & (gdp['Time'] < '2024-01-01')]

In [3]:
#Importing the Mistral Large Language Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model_name = "mistralai/Mistral-7B-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 2/2 [02:11<00:00, 65.95s/it]


In [ ]:
# Extracting relevant columns for training
gdp_train_list = gdp_train[['Time', 'Value']].values.tolist()

# Inserting gdp_train into the prompt string
prompt = f"""
You are a time series forecasting model designed to predict the Danish GDP based on historical data. 
Your task is to forecast the GDP from 2020Q1 until 2023Q4, using the provided GDP training data.

The provided GDP training data is as follows:
{gdp_train_list}

Please analyze the provided data, which represents the GDP values over time, and generate predictions for the GDP for the specified timeframe. Your output should be a list of predicted GDP values for each quarter from 2020Q1 to 2023Q4.

Note: Your predictions should consider the underlying patterns in the data and provide realistic forecasts.
"""

In [ ]:
# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate predictions
output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

# Decode and print the output
predicted_gdp = tokenizer.decode(output[0], skip_special_tokens=True)
print(predicted_gdp)



In [ ]:
print(prompt)
print(inputs)
print(output)
print(output_decoded)